In [53]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data Preprocessing

In [64]:
fire_dir = glob.glob('/content/drive/MyDrive/fire_dataset/fire_images/*.png')
non_fire_dir = glob.glob('/content/drive/MyDrive/fire_dataset/non_fire_images/*.png')

fire_dir

['/content/drive/MyDrive/fire_dataset/fire_images/fire.243.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.24.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.242.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.240.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.241.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.239.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.238.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.237.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.236.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.434.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.235.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.10.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.1.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.100.png',
 '/content/drive/MyDrive/fire_dataset/fire_images/fire.102.png',
 '/content/drive/MyDrive/fire

In [68]:
fire_df = []
non_fire_df = []
for i in fire_dir:
    fire_df.append([i,'fire'])
for j in non_fire_dir:
    non_fire_df.append([j,'non-fire'])
df = fire_df + non_fire_df
print (df)
random.shuffle(df)

[['/content/drive/MyDrive/fire_dataset/fire_images/fire.243.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.24.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.242.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.240.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.241.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.239.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.238.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.237.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.236.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.434.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.235.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.10.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fire_images/fire.1.png', 'fire'], ['/content/drive/MyDrive/fire_dataset/fir

In [69]:
data_df = pd.DataFrame(df, columns = ['path','label'])
print (data_df)

                                                  path     label
0    /content/drive/MyDrive/fire_dataset/non_fire_i...  non-fire
1    /content/drive/MyDrive/fire_dataset/fire_image...      fire
2    /content/drive/MyDrive/fire_dataset/fire_image...      fire
3    /content/drive/MyDrive/fire_dataset/fire_image...      fire
4    /content/drive/MyDrive/fire_dataset/fire_image...      fire
..                                                 ...       ...
994  /content/drive/MyDrive/fire_dataset/fire_image...      fire
995  /content/drive/MyDrive/fire_dataset/non_fire_i...  non-fire
996  /content/drive/MyDrive/fire_dataset/non_fire_i...  non-fire
997  /content/drive/MyDrive/fire_dataset/fire_image...      fire
998  /content/drive/MyDrive/fire_dataset/non_fire_i...  non-fire

[999 rows x 2 columns]


In [70]:
datagen = ImageDataGenerator(rescale=1./255,
                             height_shift_range=0.2,
                             width_shift_range=0.2,
                             horizontal_flip=True,
                             validation_split=0.2)

In [71]:
train_generator = datagen.flow_from_dataframe(data_df,
                                              x_col='path',
                                              y_col='label',
                                              images_size=(256,256),
                                              class_mode='binary',
                                              subset='training')

validation_generator = datagen.flow_from_dataframe(data_df,
                                                   x_col='path',
                                                   y_col='label',
                                                   images_size=(256,256),
                                                   class_mode='binary',
                                                   subset='validation')

Found 800 validated image filenames belonging to 2 classes.
Found 199 validated image filenames belonging to 2 classes.


### InceptionV3

In [ ]:
inception_v3 = InceptionV3(input_shape=(256,256,3),
                           weights='imagenet',
                           include_top=False)
inception_v3.trainable = False

In [ ]:
x = layers.Flatten()(inception_v3.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(1,activation='sigmoid')(x)           

model1 = Model(inception_v3.input, x) 

model1.compile(optimizer = Adam(lr=0.0001), 
               loss = 'binary_crossentropy', 
               metrics =['acc'])

In [ ]:
history = model1.fit_generator(train_generator,
                               epochs=50,
                               verbose=0,
                               validation_data=validation_generator)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
val_acc1 = model1.evaluate(validation_generator)[1]
print("validation_accuracy: " + str(val_acc1))

### EfficientNet B0

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
efficientnet_b0 = EfficientNetB0(include_top=False,
                             weights="imagenet",
                             input_shape=(256,256,3))
efficientnet_b0.trainable = False

In [ ]:
x = layers.Flatten()(efficientnet_b0.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(1,activation='sigmoid')(x)           

model2 = Model(efficientnet_b0.input, x) 

model2.compile(optimizer = Adam(lr=0.0001), 
               loss = 'binary_crossentropy', 
               metrics =['acc'])

In [ ]:
history = model2.fit_generator(train_generator,
                               epochs=50,
                               verbose=0,
                               validation_data=validation_generator)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
val_acc2 = model2.evaluate(validation_generator)[1]
print("validation_accuracy: " + str(val_acc2))

### EfficientNet B7

In [ ]:
from tensorflow.keras.applications import EfficientNetB7

In [ ]:
efficientnet_b7 = EfficientNetB7(include_top=False,
                                 weights="imagenet",
                                 input_shape=(256,256,3))
efficientnet_b7.trainable = False

In [ ]:
x = layers.Flatten()(efficientnet_b7.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(1,activation='sigmoid')(x)           

model3 = Model(efficientnet_b7.input, x) 

model3.compile(optimizer = Adam(lr=0.0001), 
               loss = 'binary_crossentropy', 
               metrics =['acc'])

In [ ]:
history = model3.fit_generator(train_generator,epochs=50,
                               verbose=0,
                               validation_data=validation_generator)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
val_acc3 = model3.evaluate(validation_generator)[1]
print("validation_accuracy: " + str(val_acc3))

### Model Comparison

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
acc = [val_acc1,val_acc2,val_acc3]
model = ['InceptionV3','EfficientNetB0','EfficientNetB7']
ax.bar(model,acc)
plt.show()